Copyright **`(c)`** 2023 Giovanni Squillero `<giovanni.squillero@polito.it>`  
[`https://github.com/squillero/computational-intelligence`](https://github.com/squillero/computational-intelligence)  
Free for personal or classroom use; see [`LICENSE.md`](https://github.com/squillero/computational-intelligence/blob/master/LICENSE.md) for details.  

# LAB9

Write a local-search algorithm (eg. an EA) able to solve the *Problem* instances 1, 2, 5, and 10 on a 1000-loci genomes, using a minimum number of fitness calls. That's all.

### Deadlines:

* Submission: Sunday, December 3 ([CET](https://www.timeanddate.com/time/zones/cet))
* Reviews: Sunday, December 10 ([CET](https://www.timeanddate.com/time/zones/cet))

Notes:

* Reviews will be assigned  on Monday, December 4
* You need to commit in order to be selected as a reviewer (ie. better to commit an empty work than not to commit)

In [1]:
from random import choices

import lab9_lib

In [2]:
from random import random, choice, randint
from functools import reduce
from collections import namedtuple
from dataclasses import dataclass
from copy import copy

from pprint import pprint

import numpy as np

In [3]:
@dataclass
class Individual:
    def __init__(self, genotype, fitness):
        self.genotype = genotype
        self.fitness = fitness

In [61]:
@dataclass
class Population:
    def __init__(self, population_size, genotype_lenght, fitness):
        self.population = []
        for _ in range(population_size):
            genotype = np.random.randint(2, size=genotype_lenght)
            self.population.append(Individual(genotype, fitness(genotype)))

    def get_best_individual(self):
        sorted_population = sorted(self.population, key= lambda i: i.fitness, reverse=True)
        best_individual = sorted_population[0]
        return best_individual
    
    def select_parent(self, tournament_size):
        pool = [choice(self.population) for _ in range(tournament_size)]
        champion = max(pool, key= lambda i: i.fitness)
        return champion
    
    def survival_selection(self, size):
        sorted_population = sorted(self.population, key= lambda i: i.fitness, reverse=True)
        self.population = sorted_population[:size]

In [16]:
POPULATION_SIZE = 50
OFFSPRING_SIZE = 20
TOURNAMENT_SIZE = 2
MUTATION_PROBABILITY = .15

In [ ]:
#Mutation & Crossover  TO DO
def mutate(ind: Individual) -> Individual:
    offspring = copy(ind)
    pos = randint(0, NUM_SETS-1)
    offspring.genotype[pos] = not offspring.genotype[pos]
    offspring.fitness = None
    return offspring

def one_cut_xover(ind1: Individual, ind2: Individual) -> Individual:
    cut_point = randint(0, NUM_SETS-1)
    offspring = Individual(fitness=None,
                           genotype=ind1.genotype[:cut_point] + ind2.genotype[cut_point:])
    assert len(offspring.genotype) == NUM_SETS
    return offspring

In [ ]:
#GA Steady State & Generationals + Elitism

### Professore

In [ ]:
for generation in range(100):
    offspring = list()
    for counter in range(OFFSPRING_SIZE):
        if random() < MUTATION_PROBABILITY:  # self-adapt mutation probability
            # mutation  # add more clever mutations
            p = select_parent(population)
            o = mutate(p)
        else:
            # xover # add more xovers
            p1 = select_parent(population)
            p2 = select_parent(population)
            o = one_cut_xover(p1, p2)
        offspring.append(o)

    for i in offspring:
        i.fitness = fitness(i.genotype)
    population.extend(offspring)
    population.sort(key=lambda i: i.fitness, reverse=True)
    population = population[:POPULATION_SIZE]
    print(population[0].fitness)

(24, -1)
(26, -1)
(40, -2)
(55, -3)
(78, -6)
(83, -7)
(85, -7)
(87, -9)
(92, -9)
(96, -11)
(98, -11)
(98, -11)
(99, -12)
(100, -15)
(100, -15)
(100, -14)
(100, -14)
(100, -14)
(100, -14)
(100, -14)
(100, -14)
(100, -14)
(100, -14)
(100, -14)
(100, -14)
(100, -14)
(100, -14)
(100, -14)
(100, -14)
(100, -14)
(100, -14)
(100, -14)
(100, -14)
(100, -14)
(100, -14)
(100, -14)
(100, -14)
(100, -14)
(100, -14)
(100, -14)
(100, -14)
(100, -14)
(100, -14)
(100, -14)
(100, -14)
(100, -14)
(100, -14)
(100, -14)
(100, -14)
(100, -14)
(100, -14)
(100, -14)
(100, -14)
(100, -14)
(100, -14)
(100, -14)
(100, -14)
(100, -14)
(100, -14)
(100, -14)
(100, -14)
(100, -14)
(100, -14)
(100, -14)
(100, -14)
(100, -14)
(100, -14)
(100, -14)
(100, -14)
(100, -14)
(100, -14)
(100, -14)
(100, -14)
(100, -14)
(100, -14)
(100, -14)
(100, -14)
(100, -14)
(100, -14)
(100, -14)
(100, -14)
(100, -14)
(100, -14)
(100, -14)
(100, -14)
(100, -14)
(100, -14)
(100, -14)
(100, -14)
(100, -14)
(100, -14)
(100, -14)
(100, -14)

In [12]:
fitness = lab9_lib.make_problem(10)
for n in range(10):
    ind = choices([0, 1], k=50)
    print(f"{''.join(str(g) for g in ind)}: {fitness(ind):.2%}")

print(fitness.calls)

11110000000001100110001001111100000001010111000100: 7.34%
10110111111111100111001001100100111111111000010101: 9.11%
10111010110000101001100001100110000101101011101111: 23.34%
01010011111110000010001110110110001001111010111011: 15.33%
11101001001100000000011100101000001111101110110011: 7.33%
01111001101110011000011011101000010111010101011000: 23.34%
11111010101100101001010010001011110101101011011011: 23.33%
00001000001000110101011110000111101001010011001011: 15.36%
00011010011010001101100000011100010000110010010001: 9.56%
10100110111000111011010110011110110011111010010001: 9.11%
10
